### <Center> Лабораторна робота №6. <br> Ідентифікація користувача за допомогою логістичної регресії

In [15]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

### 1. Завантаження і перетворення даних
Дані можна самостійно завантажити за посиланням [сторінка](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). Однак можна цього не робити, оскільки дані вже завантажені для проведення лабораторної роботи

In [3]:
# завантажимо навчальну і тестову вибірки
train_df = pd.read_csv('data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('data/test_sessions.csv',
                      index_col='session_id')

# приведемо колонку time1, ..., time10 до часового формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# відсортуємо дані за часом
train_df = train_df.sort_values(by='time1')

# подивимося на заголовок навчальної вибірки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В навчальній вибірці містяться наступні ознаки:
    - site1 – індекс першого відвідування сайту в сесії
    - time1 – час відвідування першого сайту в сесії
    - ...
    - site10 – індекс 10-го відвідування сайту в сесії
    - time10 – час відвідування 10-го сайту в сесії
    - target – цільова змінна, 1 для сесій Еліс, 0 для сесій інших користувачів
    
Сесії користувачів виділені таким чимном, щоб вони не можут бути довші півгодини чи 10 сайтів. Тобто сесія вважається закінченою або коли користувач відвідав 10 сайтів підряд або коли сесія зайняла за часом більше 30 хвилин.

В таблиці зустрічаються пропущені значення, це значить, що сесія містить менше, ніж 10 сайтів. Замінимо пропущені значення нулями і приведемо ознаки до цільового типу. Також заванатажимо словник сайтів і подивимося, як він виглядає:

In [4]:
# приведемо колонки site1, ..., site10 до цілочислового формату і замінимо пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# завантажимо словник сайтів
with open(r"data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словника сайтів
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всього сайтів:', sites_dict_df.shape[0])
sites_dict_df.head()

всього сайтів: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Виділимо цільову змінну і об'єднаємо вибірки, щоб разом привести їх до розрідженого формату.

In [5]:
# наша цільова змінна
y_train = train_df['target']

# об'єднана таблиця вхідних даних
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# індекс, за яким будемо відокремлювати навчальну вибірку від тестової
idx_split = train_df.shape[0]

Для самої першої моделі ми використовуємо лише відвідувані сайти в сесіях (але не будемо звертати увагу на часові ознаки). В основі такого вибору даних для моделей лежить така ідея: * у Еліс є свої улюблені сайти, і якщо ви ще побачите ці сайти в сесіях, тим вище ймовірність, що це сесія Еліс і навпаки. *

Підготуємо дані, з усієї таблиці виберемо лише ознаки `site1, site2, ..., site10`. Нагадуємо, що пропущені значення замінені нулем. Ось як виглядатимуть перші рядки таблиць:

In [6]:
# таблиця з індексами відвіданих сайтів в сесії
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сесії представляють собою послідовність індексів сайтів і дані в такому вигляді невдалі для лінійних методів. Відповідно до нашої гіпотези (у Еліс є улюблені сайти) необхідно перетворити цю таблицю таким чином, щоб кожен можливий веб-сайт відповідав своєму окремому призначенню (колонка), а його значення зростало за кількістю відвідувачів цього веб-сайту в сесіях. Це робиться в два рядки:

In [7]:
from scipy.sparse import csr_matrix

In [9]:
csr_matrix

scipy.sparse._csr.csr_matrix

In [10]:
# послідовність з індексами
sites_flatten = full_sites.values.flatten()

# шкана матриця
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

Ще один плюс використання розріджених матриць у тому, що для них є спеціальні реалізації як матричних операцій, так і алгоритми машинного навчання, що дозволяє істотно прискорити операції за рахунок особливостей структур даних. Це стосується і логістичної регресії. Ось тепер у нас все готове для побудови наших перших моделей.

### 2. Побудова першої моделі

Отже, у нас є алгоритм та дані для нього, побудуйте нашу першу модель, використовуючи реалізацію [логістичної регресії] (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) з пакета ` sklearn` з параметрами за замовчуванням. Перші 90% даних будемо використовувати для навчання (навчальна вибірка, відсортована за часом), а також 10% для перевірки якості (validation).

**Напишіть просту функцію, яка поверне якість моделей на вкладеній вибірці, і навчіть наш перший класифікатор**.

In [16]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – вибірка
    ratio – у якому співвідношенні поділити вибірку
    C, seed – коефіцієт регуляризації і random_state 
              логістичної регресії
    '''
    
   # Розділимо вибірку на навчальну і валідаційну в заданому співвідношенні
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=(1 - ratio), random_state=seed, shuffle=False)
    
    # Ініціалізуємо логістичну регресію з коефіцієнтом регуляризації C
    logreg = LogisticRegression(C=C, random_state=seed, solver='liblinear')
    
    # Навчаємо модель на навчальних даних
    logreg.fit(X_train, y_train)
    
    # Прогнозуємо ймовірності на валідаційній вибірці
    y_pred = logreg.predict_proba(X_valid)[:, 1]
    
    # Обчислюємо і повертаємо AUC-ROC на валідаційній вибірці
    return roc_auc_score(y_valid, y_pred)

**Подивіться, який отримано ROC AUC на відкладеній вибірці.**

In [27]:
# X - це розріджена матриця з індексами сайтів
X = full_sites_sparse[:idx_split, :]  # Навчальна вибірка (всі сесії до розділення на навчальні та тестові дані)

# y - це цільова змінна, яка вказує, чи належить сесія до Еліс (1) або ні (0)
y = y_train.values

# Навчання моделі і розрахунок AUC-ROC на валідаційній виборці
auc_score = get_auc_lr_valid(X, y, C=1.0, ratio=0.9)

# Виведення результату AUC-ROC на валідаційній вибірці
print(f'AUC-ROC на валідаційній вибірці: {auc_score:.4f}')

AUC-ROC на валідаційній вибірці: 0.9195


Будем вважати цю модель нашою першою відправною точкою (базовий рівень). Для побудови моделей для прогнозування на тестовій вибірці ** необхідно навчити модель заново вже на всій навчальній вибірці ** (покищо наша модель навчилася лише на частині даних), що підвищує її узагальнюючу здатність:

In [23]:
# функція для запису прогнозів в файлі
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Навчіть модель на всій вибірці, зробіть прогноз для тестової вибірки і покажіть результат**.

In [24]:
# Підготовка даних
X_train_full = full_sites_sparse[:idx_split, :]  # Навчальна вибірка (всі дані)
y_train_full = y_train.values  # Цільова змінна

# Тестова вибірка
X_test = full_sites_sparse[idx_split:, :]  # Тестова вибірка

# Ініціалізація логістичної регресії
logreg_full = LogisticRegression(C=1.0, random_state=17, solver='liblinear')

# Навчання на всій навчальній вибірці
logreg_full.fit(X_train_full, y_train_full)

# Прогноз для тестової вибірки
y_test_pred = logreg_full.predict_proba(X_test)[:, 1]

# Виведення перших 30 прогнозів для тестової вибірки
print([f'{pred:.4f}' for pred in y_test_pred[:30]])

# Запис прогнозів у файл
write_to_submission_file(y_test_pred, 'submission.csv')

['0.0022', '0.0000', '0.0000', '0.0000', '0.0000', '0.0002', '0.0005', '0.0001', '0.0008', '0.1031', '0.0000', '0.0001', '0.0004', '0.3577', '0.0001', '0.0040', '0.0152', '0.0000', '0.0009', '0.0000', '0.0623', '0.0000', '0.0000', '0.0003', '0.0718', '0.0000', '0.0006', '0.0000', '0.0146', '0.0001']


### 3. Покращення моделі, побудова нових ознак

Створіть таку ознаку, яка буде представлят собою число формату ГГГГММ від тої дати, коли відбувалась сесія, наприклад 201407 -- 2014 рік і 7 месяц. Таким чином, ми будемо враховувати помісячний [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь період наданих даних.

In [25]:
# Створимо нову ознаку 'year_month' на основі колонки 'time1'
# Спочатку переконаємось, що 'time1' приведена до типу datetime
train_df['time1'] = pd.to_datetime(train_df['time1'])

# Створюємо нову ознаку у форматі ГГГГММ
train_df['year_month'] = train_df['time1'].apply(lambda x: 100 * x.year + x.month)

# Те саме робимо для тестової вибірки
test_df['time1'] = pd.to_datetime(test_df['time1'])
test_df['year_month'] = test_df['time1'].apply(lambda x: 100 * x.year + x.month)

# Перевіримо, як виглядають перші кілька рядків з новою ознакою
train_df[['time1', 'year_month']].head()


,time1,year_month
session_id,,
21669,2013-01-12 08:05:57,201301
54843,2013-01-12 08:37:23,201301
77292,2013-01-12 08:50:13,201301
114021,2013-01-12 08:50:17,201301
146670,2013-01-12 08:50:20,201301


Додайте нову ознаку, попередньо нормалізуючи її за допомогою `StandardScaler`, і знову підрахуйте ROC AUC на відкладеній вибірці.

In [28]:
# Нормалізуємо цю ознаку за допомогою StandardScaler
scaler = StandardScaler()

# Застосовуємо scaler до ознаки 'year_month'
train_year_month_scaled = scaler.fit_transform(train_df[['year_month']])
test_year_month_scaled = scaler.transform(test_df[['year_month']])

# Додаємо нормалізовану ознаку до розріджених матриць X (навчальна) і X_test (тестова)
X_train_full_with_year_month = hstack([full_sites_sparse[:idx_split, :], train_year_month_scaled])
X_test_with_year_month = hstack([full_sites_sparse[idx_split:, :], test_year_month_scaled])

# Повторне навчання моделі на нових даних з новою ознакою
logreg_full_with_year_month = LogisticRegression(C=1.0, random_state=17, solver='liblinear')
logreg_full_with_year_month.fit(X_train_full_with_year_month, y_train.values)

# Прогнозування на валідаційній вибірці (знову ділимо навчальну вибірку для перевірки)
X_train, X_valid, y_train_split, y_valid = train_test_split(
    X_train_full_with_year_month, y_train.values, test_size=0.1, random_state=17, shuffle=False)

# Навчання на нових даних з валідацією
logreg = LogisticRegression(C=1.0, random_state=17, solver='liblinear')
logreg.fit(X_train, y_train_split)

# Прогнозування для валідаційної вибірки
y_valid_pred = logreg.predict_proba(X_valid)[:, 1]

# Обчислюємо AUC-ROC на валідаційній вибірці
auc_score = roc_auc_score(y_valid, y_valid_pred)
print(f'AUC-ROC на валідаційній вибірці з новою ознакою: {auc_score:.4f}')

AUC-ROC на валідаційній вибірці з новою ознакою: 0.9197


**Додайте дві нові ознаки: start_hour і morning.**

Ознака `start_hour` - це час у якому почалася сесія (від 0 до 23), а бінарна оознака` morning` рівна 1, якщо сесія почалася вранці і 0, якщо сесія почалася пізніше (будемо вважати, що це ранок, якщо `start_hour рівний` 11 або менше).

**Підрахуйте RUC AUC на відкладеній вибірці для вибірки з:**
- сайтами, `start_month` і` start_hour`
- сайтами, `start_month` і` morning`
- сайтами, `start_month`,` start_hour` і `morning`

In [31]:
# Створюємо нові ознаки 'start_hour' і 'morning' для навчальної і тестової вибірок
train_df['start_hour'] = train_df['time1'].apply(lambda x: x.hour)
train_df['morning'] = train_df['start_hour'].apply(lambda x: 1 if x <= 11 else 0)

test_df['start_hour'] = test_df['time1'].apply(lambda x: x.hour)
test_df['morning'] = test_df['start_hour'].apply(lambda x: 1 if x <= 11 else 0)

# Нормалізуємо 'year_month' і 'start_hour' за допомогою StandardScaler
scaler = StandardScaler()

# Масштабування ознак для тренувальної і тестової вибірок
train_features_for_scaling = train_df[['year_month', 'start_hour']]
test_features_for_scaling = test_df[['year_month', 'start_hour']]

train_features_scaled = scaler.fit_transform(train_features_for_scaling)
test_features_scaled = scaler.transform(test_features_for_scaling)

# Ознака 'morning' без масштабування (бінарна ознака)
train_morning = train_df[['morning']].values
test_morning = test_df[['morning']].values

# 1. Сайти + 'start_month' і 'start_hour'
X_train_1 = hstack([full_sites_sparse[:idx_split, :], train_features_scaled[:, 0].reshape(-1, 1), train_features_scaled[:, 1].reshape(-1, 1)])
X_test_1 = hstack([full_sites_sparse[idx_split:, :], test_features_scaled[:, 0].reshape(-1, 1), test_features_scaled[:, 1].reshape(-1, 1)])

# 2. Сайти + 'start_month' і 'morning'
X_train_2 = hstack([full_sites_sparse[:idx_split, :], train_features_scaled[:, 0].reshape(-1, 1), train_morning])
X_test_2 = hstack([full_sites_sparse[idx_split:, :], test_features_scaled[:, 0].reshape(-1, 1), test_morning])

# 3. Сайти + 'start_month', 'start_hour' і 'morning'
X_train_3 = hstack([full_sites_sparse[:idx_split, :], train_features_scaled[:, 0].reshape(-1, 1), train_features_scaled[:, 1].reshape(-1, 1), train_morning])
X_test_3 = hstack([full_sites_sparse[idx_split:, :], test_features_scaled[:, 0].reshape(-1, 1), test_features_scaled[:, 1].reshape(-1, 1), test_morning])

# Використовуємо функцію get_auc_lr_valid для обчислення AUC-ROC на відкладеній вибірці для кожного варіанту

# Сайти + 'start_month' і 'start_hour'
auc_score_1 = get_auc_lr_valid(X_train_1, y_train.values, C=1.0, ratio=0.9)
print(f'AUC-ROC на валідаційній вибірці для сайтів, start_month і start_hour: {auc_score_1:.4f}')

# Сайти + 'start_month' і 'morning'
auc_score_2 = get_auc_lr_valid(X_train_2, y_train.values, C=1.0, ratio=0.9)
print(f'AUC-ROC на валідаційній вибірці для сайтів, start_month і morning: {auc_score_2:.4f}')

# Сайти + 'start_month', 'start_hour' і 'morning'
auc_score_3 = get_auc_lr_valid(X_train_3, y_train.values, C=1.0, ratio=0.9)
print(f'AUC-ROC на валідаційній вибірці для сайтів, start_month, start_hour і morning: {auc_score_3:.4f}')


AUC-ROC на валідаційній вибірці для сайтів, start_month і start_hour: 0.9579
AUC-ROC на валідаційній вибірці для сайтів, start_month і morning: 0.9487
AUC-ROC на валідаційній вибірці для сайтів, start_month, start_hour і morning: 0.9591


### 4. Підбір коефіцієнта регуляризації

Отже, ми ввели ознаки, які покращують якість нашої моделі у порівнянні з першим бейслайном. Чи можемо ми домогтися більшого значення метрики? Після того, як ми сформували навчальну та тестову вибірки, майже завжди має сенс підібрати оптимальні гіперпараметри - характеристики моделі, які не змінюються під час навчання. Наприклад, ви вивчали вирішальні дерева, глибина дерева це гіперпараметр, а ознака, за якому відбувається розгалуження і її значення - ні. У використовуваної нами логістичної регресії ваги кожної ознаки змінюються і під час навчання знаходяться їх оптимальні значення, а коефіцієнт регуляризації залишається постійним. Це той гіперпараметр, який ми зараз будемо оптимізувати.

Порахуйте якість на відкладеній вибірці з коефіцієнтом регуляризації, який за замовчуванням `C = 1 ':

In [33]:
# Підрахуємо AUC-ROC на валідаційній вибірці з коефіцієнтом регуляризації C = 1
auc_score = get_auc_lr_valid(X_train_3, y_train.values, C=1.0, ratio=0.9)

# Виведемо результат
print(f'AUC-ROC на валідаційній вибірці з коефіцієнтом регуляризації C=1: {auc_score:.4f}')

AUC-ROC на валідаційній вибірці з коефіцієнтом регуляризації C=1: 0.9591


Постараємося побити цей результат за рахунок оптимізації коефіцієнта регуляризації. Візьмемо набір можливих значень C і для кожного з них порахуємо значення метрики на відкладеної вибірці.

Знайдіть `C` з` np.logspace (-3, 1, 10) `, при якому ROC AUC на відкладеної вибірці максимальний.

In [34]:
# Створимо список можливих значень C
C_values = np.logspace(-3, 1, 10)

# Змінна для зберігання найкращого значення C і відповідного AUC-ROC
best_auc = 0
best_C = None

# Перебираємо всі значення C
for C in C_values:
    # Обчислюємо AUC-ROC на валідаційній вибірці для поточного значення C
    auc_score = get_auc_lr_valid(X_train_3, y_train.values, C=C, ratio=0.9)
    
    # Виведемо результат для кожного значення C
    print(f'C={C:.4f}, AUC-ROC={auc_score:.4f}')
    
    # Якщо поточне значення AUC-ROC більше за найкраще, зберігаємо його
    if auc_score > best_auc:
        best_auc = auc_score
        best_C = C

# Виведемо найкращий результат
print(f'Найкраще значення C: {best_C:.4f}, AUC-ROC: {best_auc:.4f}')

C=0.0010, AUC-ROC=0.8239
C=0.0028, AUC-ROC=0.8934
C=0.0077, AUC-ROC=0.9398
C=0.0215, AUC-ROC=0.9564
C=0.0599, AUC-ROC=0.9605
C=0.1668, AUC-ROC=0.9611
C=0.4642, AUC-ROC=0.9603
C=1.2915, AUC-ROC=0.9586
C=3.5938, AUC-ROC=0.9557
C=10.0000, AUC-ROC=0.9513
Найкраще значення C: 0.1668, AUC-ROC: 0.9611


Нарешті, навчіть модель зі знайденим оптимальним значенням коефіцієнта регуляризації і з побудованими ознаками `start_hour`,` start_month` і `morning`. 

In [35]:
# Навчимо модель на всій навчальній вибірці зі знайденим оптимальним значенням C
best_C = 0.1668

# Використовуємо об'єднану матрицю ознак (сайти + start_month + start_hour + morning)
logreg_final = LogisticRegression(C=best_C, random_state=17, solver='liblinear')

# Навчання моделі на повній вибірці
logreg_final.fit(X_train_3, y_train.values)

# Прогноз на тестовій вибірці
y_test_pred = logreg_final.predict_proba(X_test_3)[:, 1]

# Записуємо результат у файл
write_to_submission_file(y_test_pred, 'submission_final.csv')

print("Модель навчено та результати збережено у файлі 'submission_final.csv'.")

Модель навчено та результати збережено у файлі 'submission_final.csv'.
